In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
import geopandas as gpd
from shapely.geometry import Point
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource

In [3]:
# Notebook Styling 
sns.set()
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

In [39]:
covid_deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
covid_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
covid_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [41]:
covid_confirmed_df['location'] = covid_confirmed_df.apply(lambda x: f"{x['Province/State']}, {x['Country/Region']}" if pd.notnull(x['Province/State']) else x['Country/Region'], axis=1)
locations_df = covid_confirmed_df[['location', 'Lat', 'Long']].copy()
covid_confirmed_df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1, inplace=True)
covid_confirmed_df.set_index('location', inplace=True)
covid_confirmed_df.index.name = None
covid_confirmed_df = covid_confirmed_df.T
covid_confirmed_df.reset_index(inplace=True)
covid_confirmed_df.rename({'index':'date'}, axis=1, inplace=True)
covid_confirmed_df['date'] = pd.to_datetime(covid_confirmed_df['date'], format='%m/%d/%y')
covid_confirmed_totals_df = covid_confirmed_df[[col for col in covid_confirmed_df.columns if col != 'date']].cumsum().copy()
covid_confirmed_totals_df.set_index(covid_confirmed_df['date'], inplace=True)
covid_confirmed_totals_df.reset_index(inplace=True)

In [148]:

def preprocess_covid_data(df=covid_deaths_df):
    df['location'] = df.apply(lambda x: f"{x['Province/State']}, {x['Country/Region']}" if pd.notnull(x['Province/State']) else x['Country/Region'], axis=1)
    locations_df = df[['location', 'Lat', 'Long']].copy()
    df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], axis=1, inplace=True)
    df.set_index('location', inplace=True)
    df.index.name = None
    df = df.T
    df.reset_index(inplace=True)
    df.rename({'index':'date'}, axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%m/%d/%y')
    totals_df = df[[col for col in df.columns if col != 'date']].cumsum().copy()
    totals_df.set_index(df['date'], inplace=True)
    totals_df.reset_index(inplace=True)
    return (locations_df, totals_df)

In [149]:
death_locations, deaths_df = preprocess_covid_data(df=covid_deaths_df)

In [150]:
deaths_df

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [58]:
covid_confirmed_df.head(5)

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [77]:
covid_confirmed_totals_df.head(5)

,date,Thailand,Japan,Singapore,Nepal,Malaysia,"British Columbia, Canada","New South Wales, Australia","Victoria, Australia","Queensland, Australia",Cambodia,Sri Lanka,Germany,Finland,United Arab Emirates,Philippines,India,Italy,Sweden,Spain,"South Australia, Australia",Belgium,Egypt,"From Diamond Princess, Australia",Lebanon,Iraq,Oman,Afghanistan,Bahrain,Kuwait,Algeria,Croatia,Switzerland,Austria,Israel,Pakistan,Brazil,Georgia,Greece,North Macedonia,Norway,Romania,Estonia,San Marino,Belarus,Iceland,Lithuania,Mexico,New Zealand,Nigeria,"Western Australia, Australia",Ireland,Luxembourg,Monaco,Qatar,Ecuador,Azerbaijan,Armenia,Dominican Republic,Indonesia,Portugal,Andorra,"Tasmania, Australia",Latvia,Morocco,Saudi Arabia,Senegal,Argentina,Chile,Jordan,Ukraine,Hungary,"Northern Territory, Australia",Liechtenstein,Poland,Tunisia,Bosnia and Herzegovina,Slovenia,South Africa,Bhutan,Cameroon,Colombia,Costa Rica,Peru,Serbia,Slovakia,Togo,Malta,Martinique,Bulgaria,Maldives,Bangladesh,Paraguay,"Ontario, Canada","Alberta, Canada","Quebec, Canada",Albania,Cyprus,Brunei,"Washington, US","New York, US","California, US","Massachusetts, US","Diamond Princess, US","Grand Princess, US","Georgia, US","Colorado, US","Florida, US","New Jersey, US","Oregon, US","Texas, US","Illinois, US","Pennsylvania, US","Iowa, US","Maryland, US","North Carolina, US","South Carolina, US","Tennessee, US","Virginia, US","Arizona, US","Indiana, US","Kentucky, US","District of Columbia, US","Nevada, US","New Hampshire, US","Minnesota, US","Nebraska, US","Ohio, US","Rhode Island, US","Wisconsin, US","Connecticut, US","Hawaii, US","Oklahoma, US","Utah, US",Burkina Faso,Holy See,Mongolia,Panama,"Kansas, US","Louisiana, US","Missouri, US","Vermont, US","Alaska, US","Arkansas, US","Delaware, US","Idaho, US","Maine, US","Michigan, US","Mississippi, US","Montana, US","New Mexico, US","North Dakota, US","South Dakota, US","West Virginia, US","Wyoming, US","Hubei, China",Iran,"Korea, South","France, France","Guangdong, China","Henan, China","Zhejiang, China","Hunan, China","Anhui, China","Jiangxi, China","Shandong, China","Diamond Princess, Cruise Ship","Jiangsu, China","Chongqing, China","Sichuan, China","Heilongjiang, China","Denmark, Denmark","Beijing, China","Shanghai, China","Hebei, China","Fujian, China","Guangxi, China","Shaanxi, China","Yunnan, China","Hainan, China","Guizhou, China","Tianjin, China","Shanxi, China","Gansu, China","Hong Kong, China","Liaoning, China","Jilin, China",Czechia,"Xinjiang, China","Inner Mongolia, China","Ningxia, China",Taiwan*,Vietnam,Russia,"Qinghai, China","Macau, China",Moldova,Bolivia,"Faroe Islands, Denmark","St Martin, France",Honduras,"Channel Islands, United Kingdom","New Brunswick, Canada","Tibet, China",Congo (Kinshasa),Cote d'Ivoire,"Saint Barthelemy, France",Jamaica,Reunion,Turkey,"Gibraltar, United Kingdom","Kitsap, WA, US","Solano, CA, US","Santa Cruz, CA, US","Napa, CA, US","Ventura, CA, US","Worcester, MA, US","Gwinnett, GA, US","DeKalb, GA, US","Floyd, GA, US","Fayette, GA, US","Gregg, TX, US","Monmouth, NJ, US","Burlington, NJ, US","Camden, NJ, US","Passaic, NJ, US","Union, NJ, US","Eagle, CO, US","Larimer, CO, US","Arapahoe, CO, US","Gunnison, CO, US","Kane, IL, US","Monroe, PA, US","Philadelphia, PA, US","Norfolk, VA, US","Arlington, VA, US","Spotsylvania, VA, US","Loudoun, VA, US","Prince George's, MD, US","Pottawattamie, IA, US","Camden, NC, US","Pima, AZ, US","Noble, IN, US","Adams, IN, US","Boone, IN, US","Dane, WI, US","Pierce, WI, US","Cuyahoga, OH, US","Weber, UT, US","Bennington County, VT, US","Carver County, MN, US","Charlotte County, FL, US","Cherokee County, GA, US","Collin County, TX, US","Jefferson County, KY, US","Jefferson Parish, LA, US","Shasta County, CA, US","Spartanburg County, SC, US","Harrison County, KY, US","Johnson County, IA, US","Berkshire County, MA, US","Davidson County, TN, US","Douglas County, OR, US","Fresno County, CA, US","Harford County, MD, US","Hendricks County, IN, US","Hudson County

In [151]:
df=covid_confirmed_totals_df
location='Cook County, IL, US'
case_type='Confirmed'

locations = [col for col in df.columns if col != 'date']
source = ColumnDataSource(df)
date = source.data['date']
location_data = source.data[location]
plot = figure(x_axis_type='datetime', y_axis_label=f"{case_type} COVID-19 Cases", y_axis_type='log', title=f"{case_type} Cases of COVID-19 in {location}")
line = plot.line(date, location_data)

plot.ygrid.minor_grid_line_color = 'black'
plot.ygrid.minor_grid_line_alpha = 0.1

output_notebook()

def update(location):
    line.data_source.data['y'] = source.data[location]
    push_notebook()
    
interact(update, location=locations)
_ = show(plot, notebook_handle=True)

Loading BokehJS ...

interactive(children=(Dropdown(description='location', options=('Thailand', 'Japan', 'Singapore', 'Nepal', 'Ma…

interactive(children=(Dropdown(description='location', options=('Thailand', 'Japan', 'Singapore', 'Nepal', 'Ma…

In [180]:
def plot_covid_19_data(df=covid_confirmed_totals_df, location='Cook County, IL, US', case_type='Confirmed', y_type='log'):
    locations = [col for col in df.columns if col != 'date']
    source = ColumnDataSource(df)
    date = source.data['date']
    location_data = source.data[location]
    plot = figure(x_axis_type='datetime', x_axis_label='Date', y_axis_label=f"{case_type} COVID-19 Cases", 
                  y_axis_type=y_type, title=f"{case_type} Cases of COVID-19 in {location}", plot_width=800, plot_height=500)
    line = plot.line(date, location_data)
        
    plot.title.text_font_size='18pt'
    plot.xaxis.axis_label_text_font_size = "16pt"
    plot.yaxis.axis_label_text_font_size = "16pt"
    
    plot.xaxis.axis_label_text_font_style = 'normal'
    plot.yaxis.axis_label_text_font_style = 'normal'
    
    plot.ygrid.minor_grid_line_color = 'black'
    plot.ygrid.minor_grid_line_alpha = 0.1
    
    output_notebook()
    
    def update(location):
        line.data_source.data['y'] = source.data[location]
        push_notebook()
        
    interact(update, location=locations)
    _ = show(plot, notebook_handle=True)

In [181]:
plot_covid_19_data()

Loading BokehJS ...

interactive(children=(Dropdown(description='location', options=('Thailand', 'Japan', 'Singapore', 'Nepal', 'Ma…

In [156]:
plot_covid_19_data(df=deaths_df, location='Cook County, IL, US', case_type='Fatal')

Loading BokehJS ...

interactive(children=(Dropdown(description='location', options=('Thailand', 'Japan', 'Singapore', 'Nepal', 'Ma…

In [73]:
source

ColumnDataSource(id='1006', ...)

In [82]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [83]:
x = np.linspace(0, 2*np.pi, 2000)
y = np.sin(x)

In [84]:
p = figure(title="simple line example", plot_height=300, plot_width=600, y_range=(-5,5),
           background_fill_color='#efefef')
r = p.line(x, y, color="#8888cc", line_width=1.5, alpha=0.8)

In [85]:
def update(f, w=1, A=1, phi=0):
    if   f == "sin": func = np.sin
    elif f == "cos": func = np.cos
    r.data_source.data['y'] = A * func(w * x + phi)
    push_notebook()

In [86]:
show(p, notebook_handle=True)

In [87]:
interact(update, f=["sin", "cos"], w=(0,50), A=(1,10), phi=(0, 20, 0.1))

interactive(children=(Dropdown(description='f', options=('sin', 'cos'), value='sin'), IntSlider(value=1, descr…

<function __main__.update(f, w=1, A=1, phi=0)>

In [ ]:
.apply(lambda x: Point(x['longitude'], x['latitude']), axis=1)

In [36]:
covid_confirmed_df.head(2)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20
Thailand,2,3,5,7,8,8,14,14,14,19,19,19,19,25,25,25,25,32,32,32,33,33,33,33,33,34,35,35,35,35,35,35,35,35,37,40,40,41,42,42,43,43,43,47,48,50,50,50,53,59,70,75,82,114,147
Japan,2,1,2,2,4,4,7,7,11,15,20,20,20,22,22,45,25,25,26,26,26,28,28,29,43,59,66,74,84,94,105,122,147,159,170,189,214,228,241,256,274,293,331,360,420,461,502,511,581,639,639,701,773,839,825


In [6]:
covid_confirmed_df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462
Province/State,NaN,NaN,NaN,NaN,NaN,British Columbia,New South Wales,Victoria,Queensland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,South Australia,NaN,NaN,From Diamond Princess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Western Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tasmania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Northern Territory,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ontario,Alberta,Quebec,NaN,NaN,NaN,Washington,New York,California,Massachusetts,Diamond Princess,Grand Princess,Georgia,Colorado,Florida,New Jersey,Oregon,Texas,Illinois,Pennsylvania,Iowa,Maryland,North Carolina,South Carolina,Tennessee,Virginia,Arizona,Indiana,Kentucky,District of Columbia,Nevada,New Hampshire,Minnesota,Nebraska,Ohio,Rhode Island,Wisconsin,Connecticut,Hawaii,Oklahoma,Utah,NaN,NaN,NaN,NaN,Kansas,Louisiana,Missouri,Vermont,Alaska,Arkansas,Delaware,Idaho,Maine,Michigan,Mississippi,Montana,New Mexico,North Dakota,South Dakota,West Virginia,Wyoming,Hubei,NaN,NaN,France,Guangdong,Henan,Zhejiang,Hunan,Anhui,Jiangxi,Shandong,Diamond Princess,Jiangsu,Chongqing,Sichuan,Heilongjiang,Denmark,Beijing,Shanghai,Hebei,Fujian,Guangxi,Shaanxi,Yunnan,Hainan,Guizhou,Tianjin,Shanxi,Gansu,Hong Kong,Liaoning,Jilin,NaN,Xinjiang,Inner Mongolia,Ningxia,NaN,NaN,NaN,Qinghai,Macau,NaN,NaN,Faroe Islands,St Martin,NaN,Channel Islands,New Brunswick,Tibet,NaN,NaN,Saint Barthelemy,NaN,NaN,NaN,Gibraltar,"Kitsap, WA","Solano, CA","Santa Cruz, CA","Napa, CA","Ventura, CA","Worcester, MA","Gwinnett, GA","DeKalb, GA","Floyd, GA","Fayette, GA","Gregg, TX","Monmouth, NJ","Burlington, NJ","Camden, NJ","Passaic, NJ","Union, NJ","Eagle, CO","Larimer, CO","Arapahoe, CO","Gunnison, CO","Kane, IL","Monroe, PA","Philadelphia, PA","Norfolk, VA","Arlington, VA","Spotsylvania, VA","Loudoun, VA","Prince George's, MD","Pottawattamie, IA","Camden, NC","Pima, AZ","Noble, IN","Adams, IN","Boone, IN","Dane, WI","Pierce, WI","Cuyahoga, OH","Weber, UT","Bennington County, VT","Carver County, MN","Charlotte County, FL","Cherokee County, GA","Collin County, TX","Jefferson County, KY","Jefferson Parish, LA","Shasta County, CA","Spartanburg County, SC","Harrison County, KY","Johnson Coun